In [1]:
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

import os
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image, ImageDraw
import random

Using TensorFlow backend.


In [2]:
#Random start of population
#genomes as a list of gene sequences
def initialise():
    genomes =[]
    for i in range(2):
        genomes.append([np.random.randint(2, size=6)]) # add 5 randomly initialised indiviudals


#     print (genomes)
    
    return genomes

    

In [3]:
#usefull functions

def paring(mom,dad):
    return(np.concatenate((mom[:3], dad[3:]), axis=0))


def mutation(popul):
    position = random.randrange(0, len(popul))
    gencode = popul[position]
    
    decider = random.uniform(0, 1)
    posi = 0
    if decider < 0.2 :
        print('Mutation')
        posi = random.randrange(0, len(gencode))
    
        if gencode[posi] == 1:
            gencode[posi] = 0
        elif gencode[posi] == 0:
            gencode[posi] = 1
            
    popul[position] = np.array(gencode)
    
    return popul


def selected(maxis, actualpop):
    childs= []
    maxis= np.array(maxis)
    indices = maxis.argsort()[-2:][::-1]
    print('Highest acc at')
    print(indices)
    parents= []
    for ind in indices:
        r = 0
        for k in actualpop[ind]:
         
            k=list(k)
            parents.append(k)
            r = r+1
   
    
    random.shuffle(parents)
    f= 0
    while f < len(parents)-1: 
        mom = parents[f]
        f= f +1
        dad = parents[f]     
        child = paring(mom,dad)
        childs.append(child)
        child = paring(dad,mom)
        childs.append(child)
        
        
    return childs


In [4]:
def netzdurchlauf(acutalpop):
    accuracy_of_population= []
    
    print('Aktuelle Population')
    print(actualpop)
    for element in actualpop:
        print('Element')
        print(element)
        





        batch_size = 128
        num_classes = 10
        epochs = 12

        # input image dimensions
        img_rows, img_cols = 28, 28

        # the data, split between train and test sets
        (x_train, y_train), (x_test, y_test) = mnist.load_data()

        if K.image_data_format() == 'channels_first':
            x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
            x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
            input_shape = (1, img_rows, img_cols)
        else:
            x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
            x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
            input_shape = (img_rows, img_cols, 1)

        x_train = x_train.astype('float32')
        x_test = x_test.astype('float32')
        x_train /= 255
        x_test /= 255
#             print('x_train shape:', x_train.shape)
#             print(x_train.shape[0], 'train samples')
#             print(x_test.shape[0], 'test samples')

        # convert class vectors to binary class matrices
        y_train = keras.utils.to_categorical(y_train, num_classes)
        y_test = keras.utils.to_categorical(y_test, num_classes)

        model = Sequential()




        model.add(Conv2D(32, kernel_size=(3, 3),activation='relu',input_shape=input_shape))
        model.add(Conv2D(64, (3, 3), activation='relu'))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.25))
            ########
            
            

        for gen in element:
            if (gen== 0):
                model.add(Conv2D(64, (3, 3), activation='relu'))
            elif (gen == 1):
                model.add(Dense(128, activation='relu'))




        model.add(Flatten())
        model.add(Dropout(0.5))
        model.add(Dense(num_classes, activation='softmax'))

        model.compile(loss=keras.losses.categorical_crossentropy,
                          optimizer=keras.optimizers.Adadelta(),
                          metrics=['accuracy'])

        model.fit(x_train, y_train,
                      batch_size=batch_size,
                      epochs=epochs,
                      verbose=1,
                      validation_data=(x_test, y_test))
        score = model.evaluate(x_test, y_test, verbose=0)
        print('Test loss:', score[0])
        print('Test accuracy:', score[1])
        accuracy_of_population.append(score[1])
        
    return (accuracy_of_population)

In [5]:

# umschreiben ist nur der erste durchlauf

def netzdurchlauf1(acutalpop):
    accuracy_of_population= []
    
    print('Aktuelle Population')
    print(actualpop)
    for element in actualpop:
#         print('Element')
#         print(element)
        for indiv in element:
#             print(indiv)





            batch_size = 128
            num_classes = 10
            epochs = 12

            # input image dimensions
            img_rows, img_cols = 28, 28

            # the data, split between train and test sets
            (x_train, y_train), (x_test, y_test) = mnist.load_data()

            if K.image_data_format() == 'channels_first':
                x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
                x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
                input_shape = (1, img_rows, img_cols)
            else:
                x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
                x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
                input_shape = (img_rows, img_cols, 1)

            x_train = x_train.astype('float32')
            x_test = x_test.astype('float32')
            x_train /= 255
            x_test /= 255
#             print('x_train shape:', x_train.shape)
#             print(x_train.shape[0], 'train samples')
#             print(x_test.shape[0], 'test samples')

            # convert class vectors to binary class matrices
            y_train = keras.utils.to_categorical(y_train, num_classes)
            y_test = keras.utils.to_categorical(y_test, num_classes)

            model = Sequential()




            model.add(Conv2D(32, kernel_size=(3, 3),
                             activation='relu',
                             input_shape=input_shape))
            model.add(Conv2D(64, (3, 3), activation='relu'))
            model.add(MaxPooling2D(pool_size=(2, 2)))
            model.add(Dropout(0.25))
            ########
            
            

            for gen in indiv:
                if (gen== 0):
                    model.add(Conv2D(64, (3, 3), activation='relu'))
                elif (gen == 1):
                    model.add(Dense(128, activation='relu'))




            model.add(Flatten())
            model.add(Dropout(0.5))
            model.add(Dense(num_classes, activation='softmax'))

            model.compile(loss=keras.losses.categorical_crossentropy,
                          optimizer=keras.optimizers.Adadelta(),
                          metrics=['accuracy'])

            model.fit(x_train, y_train,
                      batch_size=batch_size,
                      epochs=epochs,
                      verbose=1,
                      validation_data=(x_test, y_test))
            score = model.evaluate(x_test, y_test, verbose=0)
            print('Test loss:', score[0])
            print('Test accuracy:', score[1])
            accuracy_of_population.append(score[1])
        
    return (accuracy_of_population)



In [6]:
# print("NEW Population :")
# actualpop= selected(accuracy_of_population)
# print(actualpop)
# actualpop = mutation(actualpop)
# print(actualpop)


In [ ]:

# Hauptmethode

DURCHGÄNGE = 2 # variable to clarify number of generations
actualpop= initialise()
evalaccuris = []

i = 0
if i == 0:
    accuri = netzdurchlauf1(actualpop)
    print(' Accuracy of first population')
    print(accuri)
    print('highest accuracy of first population:')
    print(numpy.amax(accuri))
    actualpop = selected(accuri, actualpop)
    i= i+1
    evalaccuris.append([i +1 ,numpy.amax(accuri)])
    
    
while i < DURCHGÄNGE:
    nummer = i
    accuri = netzdurchlauf(actualpop)
    print(' Accuracy of populationnummer: ' + nummer)
    print(accuri)
    print('highest accuracy of populationnummer: ' + nummer)
    print(numpy.amax(accuri))
    
    
    actualpop = selected(accuri, actualpop)
    i = i+1
    evalaccuris.append([i +1 ,numpy.amax(accuri)])
    
    
    
    
print('The highest in population')
print(evalaccuris)




[[array([1, 0, 0, 0, 0, 1])], [array([0, 1, 0, 1, 1, 0])]]
Aktuelle Population
[[array([1, 0, 0, 0, 0, 1])], [array([0, 1, 0, 1, 1, 0])]]


W0914 13:41:29.867754 11344 deprecation_wrapper.py:119] From C:\Users\vivia\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0914 13:41:29.887728 11344 deprecation_wrapper.py:119] From C:\Users\vivia\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0914 13:41:29.895718 11344 deprecation_wrapper.py:119] From C:\Users\vivia\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0914 13:41:29.951644 11344 deprecation_wrapper.py:119] From C:\Users\vivia\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W0914 13:41:29.955638 11344 deprecation_wrapper.py:119] From C:\Users\vivia\Ana

Train on 60000 samples, validate on 10000 samples
Epoch 1/12
60000/60000 [==============================] - 42s 701us/step - loss: 0.3452 - acc: 0.8877 - val_loss: 0.0618 - val_acc: 0.9805
Epoch 2/12
60000/60000 [==============================] - 34s 574us/step - loss: 0.0741 - acc: 0.9779 - val_loss: 0.0464 - val_acc: 0.9847
Epoch 3/12
60000/60000 [==============================] - 33s 551us/step - loss: 0.0478 - acc: 0.9856 - val_loss: 0.0289 - val_acc: 0.9910
Epoch 4/12
60000/60000 [==============================] - 34s 573us/step - loss: 0.0383 - acc: 0.9881 - val_loss: 0.0212 - val_acc: 0.9925
Epoch 5/12
60000/60000 [==============================] - 33s 546us/step - loss: 0.0316 - acc: 0.9901 - val_loss: 0.0207 - val_acc: 0.9940
Epoch 6/12
60000/60000 [==============================] - 33s 555us/step - loss: 0.0264 - acc: 0.9917 - val_loss: 0.0237 - val_acc: 0.9919
Epoch 7/12
60000/60000 [==============================] - 34s 571us/step - loss: 0.0235 - acc: 0.9930 - val_loss: 0.

In [ ]:
# print(accuri)
# print(selected(accuri, actualpop))
# actualpop = selected(accuri, actualpop)
# actualpop = mutation(actualpop)


# print('tests')
# print(actualpop)
# print( np.array(actualpop))

# print(list(actualpop))


# print()


# accuri = netzdurchlauf(actualpop)

# accuri = netzdurchlaufs(actualpop)
# actualpop = selected(accuri, actualpop)
# actualpop = mutation(actualpop)
# accuri = netzdurchlauf(actualpop)


